C:\Program Files\Graphviz\bin>dot -Tpdf C:/Users/student/Downloads/loco_tree.dot -o "C:/Users/student/Downloads/loco_tree.pdf"

**IMPORTING LIBRARIES**

To import the correct libraries, we first had to pip install the spotipy library, to extract data from the spotify API, and the requests library, to get the authorization keys and run them to authenticate the spotify developer account so we could connect to the API.

From there, we imported the necessary libraries for this project.

In [1]:
! pip install spotipy

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 
import os
import sys
import json
import webbrowser 
import spotipy.util as util
from json.decoder import JSONDecodeError
import base64

**Auhtorization for the Spotify API**

To connect to the Spotify API, we had to connect to the API by using our spotify credentials (spotify_id, spotify_secret) in the function below to allow this code to extract spotify data from the API. 

In [3]:
spotify_id = '66bc255077d74940a6272a0d2601e6d9'
spotify_secret = 'ffeb8bd48e9a4ec3a68a9749ece515a3'
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(spotify_id,spotify_secret))

**EXTRACTING DATA**

To make extraction easier for me, I created playlists in spotify for all of the past Locura de Marzo tournaments and put them in order by seed. However, there was one song that Spotify did not have (No Tengo Dinero). We had to keep that in mind when running the simulation. 

To get the song ids for each song, we used the getTrackIDs function* to extract every song id from each playlist. All of the lists in id_list contain every song for that year's tournament. From there, we appended each list created by the getTrackIDs function into one list (playlists) so one list would be an item in the playlists list. From there, we extracted each list from playlists and assigned each their own variable to put into a list (playlist_list). The lists in the playlist_list hold the song ids of each tournament.

From there, we used the getTrackFratures function*, which extracts song metadata and returns it as a list. the variable, track_head, is a list that was needed to name columns in the dataframes we are going to make later.

*I credit Sander Horpestad with this code (https://morioh.com/p/31b8a607b2b0)

In [4]:
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

In [5]:
id_oct = '7flePFOJJdH9movvNzwMsF'
id_17 = '6xWcKO4TZZ4f5UMrxGgLkv'
id_18 = '0uhlhWI8qBI1cn9L4Ibrdd'
id_19 = '59D6faNL4KGt2R6Jg6U7F9'
id_20 = '57mPJ0uEewNBk5RXyTQWkc'

id_list = [id_oct, id_17, id_18, id_19, id_20]

In [6]:
playlists = []
for x in id_list:
    playlists.append(getTrackIDs('3kiizy8ug80fzb41x3bdhq931', x))

In [7]:
playlist_oct = playlists[0]
playlist_17 = playlists[1]
playlist_18 = playlists[2]
playlist_19 = playlists[3]
playlist_20 = playlists[4]

playlist_list = [playlist_oct, playlist_17, playlist_18, playlist_19, playlist_20]

In [8]:
def getTrackFeatures(id):
    meta = sp.track(id)
    features = sp.audio_features(id)

  # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

    # features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    key = features[0]['key']
    mode = features[0]['mode']
    time_signature = features[0]['time_signature']

    track = [name, album, artist, release_date, length, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, key, mode, time_signature]
    return track

In [9]:
track_head = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability',
         'acousticness', 'energy', 'instrumentalness', 'liveness', 
         'loudness', 'speechiness', 'tempo', 'key', 'mode', 'time_signature']

**Creating Dataframes**

To create the dataframes, we repeated the same function for each dataframe because the we could not find a way to create a for loop that could do what we wanted it to do. For we created a track list (track_xx) that is initially empty at first. The for loop uses the playlist list (playlist_xx) of the tournament we are creating the dataframe for. From there, we ran each song id in the playlist list in the getTrackFeatures function to get all the data we needed for each song in spotify. We appended that data into the track_list. Finally, we created a dataframe for that tournament using the fully appended track list as the data and the list of names in track_head as the column names.

The only issue we ran into was that for the df_20, there were two too many songs, so the last function before the next markdown takes out the two that lost in the preliminary round and resets the dataframe index.

In [10]:
track_oct = []
for i in range(len(playlist_oct)):
    track = getTrackFeatures(playlist_oct[i])
    track_oct.append(track)

df_oct = pd.DataFrame(track_oct, columns = track_head)

In [11]:
track_17 = []
for i in range(len(playlist_17)):
    track = getTrackFeatures(playlist_17[i])
    track_17.append(track)

df_17 = pd.DataFrame(track_17, columns = track_head)

In [12]:
track_18 = []
for i in range(len(playlist_18)):
    track = getTrackFeatures(playlist_18[i])
    track_18.append(track)

df_18 = pd.DataFrame(track_18, columns = track_head)

In [13]:
track_19 = []
for i in range(len(playlist_19)):
    track = getTrackFeatures(playlist_19[i])
    track_19.append(track)

df_19 = pd.DataFrame(track_19, columns = track_head)

In [14]:
track_20 = []
for i in range(len(playlist_20)):
    track = getTrackFeatures(playlist_20[i])
    track_20.append(track)

df_20 = pd.DataFrame(track_20, columns = track_head)

In [15]:
df_20 = df_20.drop(df_20.index[[2,3]])
df_20 = df_20.reset_index()

**Adding Calculated Columns**

The dataframe had the information we wanted but not in the way we wanted it to be Therefore, we had to create calculated columns to reformat the data. 

- LENGTH: Initially, the length of the song was measured in milliseconds (length). To gage this better, we wanted to length to be calculated by minutes. To do this, we create the column "minutes" by dividing the length by 60000: the conversion rate of minutes to milliseconds.
- SEED: Because I organized the songs in my playlist by seed, the code above imported the songs in that order. The one seed was in the first row of the dataframe , but its index was 0. To correctly get the seed, I used the index of the song in the dataframe and added it by 1. 
- YEAR: using the DatetimeIndex function, we extracted the year out of the release date to get its release year. 
- SHARP: The key of the song is organized by numbers from 0-11. With these keys, however, there are major and minor variations, which I found was in the metadata under a variable called mode (0 for minor, 1 for major). In order to get a better representation of the key, I organized every major and minor key based on the number of black keys in its scale. Using a chart called "the circle of fifths", I found out which major and minor keys had how many black keys. With that information, I created the black_keys function to identify each key and tell how many sharps are in said key.

In [16]:
def black_keys(x):
    if (x['key'] == 0 and x['mode'] == 1) or  (x['key'] == 9 and x['mode'] == 0): 
        return 0 
    elif (((x['key'] == 1 or x['key'] == 6) or x['key'] == 11) and x['mode'] == 1) or (((x['key'] == 3 or x['key'] == 8) or x['key'] == 10) and x['mode'] == 0): 
        return 5
    elif ((x['key'] == 2 or x['key'] == 10) and x['mode'] == 1) or ((x['key'] == 7 or x['key'] == 11) and x['mode'] == 0): 
        return 2
    elif ((x['key'] == 3 or x['key'] == 9) and x['mode'] == 1) or ((x['key'] == 0 or x['key'] == 6) and x['mode'] == 0): 
        return 3
    elif ((x['key'] == 4 or x['key'] == 8) and x['mode'] == 1) or ((x['key'] == 1 or x['key'] == 5) and x['mode'] == 0): 
        return 4 
    elif ((x['key'] == 5 or x['key'] == 7) and x['mode'] == 1) or ((x['key'] == 4 or x['key'] == 2) and x['mode'] == 0): 
        return 1

In [17]:
df_list = [df_oct, df_17, df_18, df_19, df_20]
for x in df_list:
    x['minutes']= x.length/60000
    x['seed'] = x.index + 1
    x['year'] = pd.DatetimeIndex(x['release_date']).year
    x['sharp'] = x.apply(black_keys, axis=1)

**MOCK SIMULATIONS**

In these trials, there were many errors, but each served its purpose in creating the final simulation.

Simulation 1:

For our first attempt, we created a dictionary that used the name and a list that used tempo as the one variable to predict the winner. The data dictionary is used match each tempo to the song for the end from the dataframe. The list (rankings) extracted data from the tempo column in the dataframe and places them in order of seed. 

The first for loop simulates each round. The wins variable is a list that gets redefined each round to hold the winners of each round. From there, I created a variable y, which determines how many matchups there are in each round (i.e. in the first round, there are 16 songs, represented by the 16 tempos in the rankings list). 

The nested for loop used y to create the matchups. The variable h represents the higher seed in the matchup and the variable l represents the lower seed. h is set to equal to the first item in rankings (ranking[0]) at first and l is represented by the last item in rankings (ranking[-0-1] = ranking[-1]). I then print the two tempos that match up in the round. The if functions below determine the winner. I made it so the song with the fastest tempo would win. The winning song would be appended to the wins list. If there were a tie, I said it was a push. 

I then set rankings to equal wins so when the first for loop begins again, it works with the updated list of the next round.

The way we created the for loop that creates the matchup mimics the way the standard brackets play out. In bracket-style matchups, the winner of the 1 vs 16 matchup always plays the winner of the 8 vs 9 matchup. Our for loop works from the outside in. The first item matchs up with the last item, and because these items are in order of seed, that make that first matchup 1 vs 16. From there, the loop continues by moving its way to the center of the list. The next matchup would be the second item in rankings (the 2 seed, rankings[1]) vs the second to last item in rankings (the 15 seed, rankings[-1-1] = ranking[-2]). The last matchup that is ran in the first round is the 8 seed vs 9 seed (ranking[y] vs ranking[-y-1]). The winner of that matchup would be the last item appended into the wins list. When the rankings list is set to equal the wins list at the end of each round, the winner of the 8 vs 9 matchup become the last item in rankings. Since the winner of the 1 vs 16 matchup was appended first to wins, in the second round, the first item in the new rankings list (winner of 1 vs 16) will face the last item in the list (winner of 8 vs 9) no matter what the outcome of the previous matchups were.

When there is only one team left in rankings, I print the name of the song based on the dictionary (dict1) I created.

In [18]:
dict1=pd.Series(df_oct.name.values,index = df_oct.tempo).to_dict()

In [19]:
rankings = df_oct.tempo.to_list()

In [20]:
for q in range(4):
    wins = []
    y = len(rankings)/2
    y = int(y)
    for x in range(0, y):
        h = rankings[x]
        l = rankings[-x-1]
        print(h, ' vs. ', l)
        if l < h: 
            wins.append(h)
        if l > h:
            wins.append(l)
        if h == l: 
            print('push')
    print(wins)
    rankings = wins
rankings

126.988  vs.  124.014
122.027  vs.  95.993
95.046  vs.  117.039
112.004  vs.  115.964
105.017  vs.  89.911
104.114  vs.  95.989
94.931  vs.  112.991
93.832  vs.  104.959
[126.988, 122.027, 117.039, 115.964, 105.017, 104.114, 112.991, 104.959]
126.988  vs.  104.959
122.027  vs.  112.991
117.039  vs.  104.114
115.964  vs.  105.017
[126.988, 122.027, 117.039, 115.964]
126.988  vs.  115.964
122.027  vs.  117.039
[126.988, 122.027]
126.988  vs.  122.027
[126.988]


[126.988]

In [21]:
print(dict1[rankings[0]])

Waka Waka (This Time for Africa) [The Official 2010 FIFA World Cup (TM) Song] (feat. Freshlyground)


Simulation 2:

One of the glaring issues for the simulation above was that there was too much middleman work in the form of the list and the dictionary. To fix this, we simulated only one round, but for my variables h and l, we were able to extract all the songs' attributes by using the iloc[] function on the dataframe in the format of a dataframe row. The wins list function stayed the same, but the variable y had to change from measuring the length of a list to the length of a dataframe. All that had to be done was put the dataframe name into the len() function to say how many rows there were. The if functions now extract the desired variables by inputting their column name. I used seed because it is very easy to track. To make the winners easier to track, for the winners who were appended to the wins list, we printed the name of the winning song.

In [22]:
df_headers = list(df_oct.columns)
df_r1 = df_oct
df_r2 = pd.DataFrame(columns = df_headers)

In [23]:
wins = []
y = len(df_r1)/2
y = int(y)
for x in range(0, y):
    h = df_r1.iloc[x]
    l = df_r1.iloc[-x-1]
    print(h['name'], ' vs. ', l['name'])
    if h['seed'] < l['seed']: 
        wins.append(h)
        print("Winner:", h['name'])
    if h['seed'] > l['seed']:
        wins.append(l)
        print("Winner:", l['name'])
df_r2 = pd.DataFrame(data = wins, columns = df_headers)

Waka Waka (This Time for Africa) [The Official 2010 FIFA World Cup (TM) Song] (feat. Freshlyground)  vs.  Yo Voy Ganao
Winner: Waka Waka (This Time for Africa) [The Official 2010 FIFA World Cup (TM) Song] (feat. Freshlyground)
Lo Mismo  vs.  Cuando Nadie Ve
Winner: Lo Mismo
Robarte un Beso  vs.  Soy Yo
Winner: Robarte un Beso
La Libertad  vs.  Baila
Winner: La Libertad
Vivir Mi Vida  vs.  Somos los Prietos (feat. Alexis Play)
Winner: Vivir Mi Vida
Andas En Mi Cabeza  vs.  Échame La Culpa
Winner: Andas En Mi Cabeza
Un Año  vs.  El Mismo Sol
Winner: Un Año
Cásate Conmigo  vs.  Mi Gente
Winner: Cásate Conmigo


Simulation 3:

The previous simulation only simulated one round. To create an accurate simulation, we had to create multiple dataframes that can be appended with data for the new round and place them in a list (rounds). For each of the dataframes, they matched the same headers as the first dataframe (df_headers). Each dataframe was put into a list ordered by the round that they corresponded to.

I added a for loop that repeats 4 times (one for each round) and we replaced every instance of df_r1 in the code above with "round[r]". For the final append step, We used the wins list as data in the new dataframe with the column names equal to df_header. To keep records of the songs in each round, I make the dataframe created in the last step equal to the next item in rounds.

The final step was to take the items in the rounds list and set them equal to themselves (df_r1 = rounds[0], etc)

In [24]:
df_headers = list(df_oct.columns)

In [25]:
df_r1 = df_oct
df_r2 = pd.DataFrame(columns = df_headers)
df_r3 = pd.DataFrame(columns = df_headers)
df_r4 = pd.DataFrame(columns = df_headers)
df_winner = pd.DataFrame(columns = df_headers)

rounds = [df_r1, df_r2, df_r3, df_r4, df_winner]

In [26]:
rounds = [df_r1, df_r2, df_r3, df_r4, df_winner]
for r in range (0,4):
    wins = []
    y = len(rounds[r])/2
    y = int(y)
    for x in range(0, y):
        h = rounds[r].iloc[x]
        l = rounds[r].iloc[-x-1]
        print(h['name'], ' vs. ', l['name'])
        if h['tempo'] > l['tempo']: 
            wins.append(h)
            print("Winner:", h['name'])
        if h['tempo'] < l['tempo']:
            wins.append(l)
            print("Winner:", l['name'])
    rounds[r+1] = pd.DataFrame(data = wins, columns = df_headers)

Waka Waka (This Time for Africa) [The Official 2010 FIFA World Cup (TM) Song] (feat. Freshlyground)  vs.  Yo Voy Ganao
Winner: Waka Waka (This Time for Africa) [The Official 2010 FIFA World Cup (TM) Song] (feat. Freshlyground)
Lo Mismo  vs.  Cuando Nadie Ve
Winner: Lo Mismo
Robarte un Beso  vs.  Soy Yo
Winner: Soy Yo
La Libertad  vs.  Baila
Winner: Baila
Vivir Mi Vida  vs.  Somos los Prietos (feat. Alexis Play)
Winner: Vivir Mi Vida
Andas En Mi Cabeza  vs.  Échame La Culpa
Winner: Andas En Mi Cabeza
Un Año  vs.  El Mismo Sol
Winner: El Mismo Sol
Cásate Conmigo  vs.  Mi Gente
Winner: Mi Gente
Waka Waka (This Time for Africa) [The Official 2010 FIFA World Cup (TM) Song] (feat. Freshlyground)  vs.  Mi Gente
Winner: Waka Waka (This Time for Africa) [The Official 2010 FIFA World Cup (TM) Song] (feat. Freshlyground)
Lo Mismo  vs.  El Mismo Sol
Winner: Lo Mismo
Soy Yo  vs.  Andas En Mi Cabeza
Winner: Soy Yo
Baila  vs.  Vivir Mi Vida
Winner: Baila
Waka Waka (This Time for Africa) [The Official

In [27]:
df_r1 = rounds[0]
df_r2 = rounds[1]
df_r3 = rounds[2]
df_r4 = rounds[3]
df_winner = rounds[4]

In [29]:
df_winner

,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,...,loudness,speechiness,tempo,key,mode,time_signature,minutes,seed,year,sharp
0,Waka Waka (This Time for Africa) [The Official...,Waka Waka (This Time for Africa) [The Official...,Shakira,2010-05-07,202626,77,0.766,0.00627,0.873,0.0,...,-6.381,0.143,126.988,11,0,4,3.3771,1,2010,2
